In [1]:
#I got this code from 'project detail' in Data Wrangling course.
#And I revised a little to fit for Python3 and my computer.
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.cElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "C:\\projects\\New_York_OSM\\New_York.osm"  # Replace this with your osm file
SAMPLE_FILE = "C:\\projects\\New_York_OSM\\New_York_sample.osm"

k = 30 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    print(root.tag, root.attrib)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n'.encode('utf-8'))
    output.write('<osm>\n  '.encode('utf-8'))

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>'.encode('utf-8'))

osm {'version': '0.6', 'generator': 'osmconvert 0.8.5', 'timestamp': '2017-09-03T15:00:02Z'}


In [7]:
#I got this code from Data Wrangling course.
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

#import cerberus

#import schema

OSM_PATH = "C:\\projects\\New_York_OSM\\New_York_sample.osm"

NODES_PATH = "C:\\projects\\New_York_OSM\\sample_nodes.csv"
NODE_TAGS_PATH = "C:\\projects\\New_York_OSM\\sample_nodes_tags.csv"
WAYS_PATH = "C:\\projects\\New_York_OSM\\sample_ways.csv"
WAY_NODES_PATH = "C:\\projects\\New_York_OSM\\sample_ways_nodes.csv"
WAY_TAGS_PATH = "C:\\projects\\New_York_OSM\\sample_ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

#SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    if element.tag == 'node':
        for key, value in element.attrib.items():
            if key in node_attr_fields:
                if key == 'id':
                    nodeId = value
                node_attribs[key] = value
            #print('node')
        for sub in element:
            key, value = sub.attrib['k'], sub.attrib['v']
            #print 'node tag', key, value
            if not problem_chars.search(key):
                
                if LOWER_COLON.match(key):
                    left, right = key.split(":", 1)
                    tags.append({"id" : nodeId, "key" : right, "value" : value, "type" : left})
                    #print('node tag', left, right, value)
                else:
                    tags.append({"id" : nodeId, "key" : key, "value" : value, "type" : default_tag_type})
                    #print('node tag', default_tag_type, key, value)
        #print node_attribs, tags        
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        for key, value in element.attrib.items():
            if key in WAY_FIELDS:
                if key == 'id':
                    wayId = value
                way_attribs[key] = value
                #print('way')
        pos = 0
        for sub in element:
            if sub.tag == 'tag':
                key, value = sub.attrib['k'], sub.attrib['v']
                if not problem_chars.search(key):
                    
                    if LOWER_COLON.match(key):
                        left, right = key.split(":", 1)
                        tags.append({"id" : wayId, "key" : right, "value" : value, "type" : left})
                        #print('way tag', left, right, value)
                    else:
                        tags.append({"id" : wayId, "key" : key, "value" : value, "type" : default_tag_type})
                        #print('way tag', default_tag_type, key, value)
            else:
                ref = sub.attrib['ref']
                way_nodes.append({"id" : wayId, "node_id" : ref, "position" : pos})
                #print('way nodes', ref, pos)
                pos += 1
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=None):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

'''
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)
'''

# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with open(NODES_PATH, 'w', encoding='UTF-8', newline='') as nodes_file, \
         open(NODE_TAGS_PATH, 'w', encoding='UTF-8', newline='') as nodes_tags_file, \
         open(WAYS_PATH, 'w', encoding='UTF-8', newline='') as ways_file, \
         open(WAY_NODES_PATH, 'w', encoding='UTF-8', newline='') as way_nodes_file, \
         open(WAY_TAGS_PATH, 'w', encoding='UTF-8', newline='') as way_tags_file:

        nodes_writer = csv.DictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = csv.DictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = csv.DictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = csv.DictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = csv.DictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        #validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                #if validate is True:
                #    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=False)

In [2]:
#Connect to sampleosm.db using sqlite3.
#First, query what are the keys of tags.
import sqlite3

db = sqlite3.connect("C:\\Users\\JehyeonHeo\\sqlite_windows\\sampleosm.db")
c = db.cursor()
query = "select distinct tags.key as key \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        order by key;"
c.execute(query)
rows = c.fetchall()
for row in rows:
    print(row)

('Bergen_County_database_ref',)
('FIXME',)
('Fax',)
('HFCS',)
('NHD:AUXCODE',)
('NHD:BldgAddr_1',)
('NHD:BldgName_1',)
('NHD:BldgNum',)
('NHD:CampusCo_1',)
('NHD:City_1',)
('NHD:ComID',)
('NHD:FCode',)
('NHD:FDate',)
('NHD:FTYPE',)
('NHD:FType',)
('NHD:Lat_1',)
('NHD:Long_1',)
('NHD:RESOLUTION',)
('NHD:ReachCode',)
('NHD:State_1',)
('NHD:Zip_1',)
('NHD:way_id',)
('NHS',)
('NJDOT_SRI',)
('NOTE',)
('NYDEC_Lands:CATEGORY',)
('NYDEC_Lands:COUNTY',)
('NYDEC_Lands:FACILITY',)
('abandoned',)
('access',)
('admin_level',)
('aerialway',)
('aeroway',)
('agricultural',)
('alt_name',)
('amenity',)
('area',)
('artist_name',)
('artwork_type',)
('atc',)
('atm',)
('attraction',)
('attribution',)
('auxcode',)
('backrest',)
('backward',)
('bar',)
('barrier',)
('basin',)
('beauty',)
('bell',)
('bench',)
('bicycle',)
('bicycle_parking',)
('bidirectional',)
('bin',)
('bitcoin',)
('black_owned',)
('board_type',)
('boat',)
('border_type',)
('both_ways',)
('boundary',)
('boundary_type',)
('brand',)
('bridge',)